<h1> TRAIN SCHEME </h1>

In [1]:
import time
start1 = time.time()

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.feature import local_binary_pattern
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

<h1>2. Deklarasi Fungsi Load Dataset</h1>

In [2]:
def load_dataset(folderdataset):
    image = []
    label = []
    folders = os.listdir(os.path.dirname(folderdataset))
    noLabel = 0
    grannysmith = 0
    for f in folders:
        if f!='.DS_Store':
            path = folderdataset+f
            for filename in os.listdir(path):
                if filename !='.DS_Store':
                    file = path+'/'+filename
                    img = np.asarray(cv2.imread(file))
                    image.append(img)
                    label.append(f)
#         noLabel = noLabel+1
    return image,label

<h1>3. Deklarasi Fungsi Preprocessing</h1>

In [3]:
def preprocessing_texture(array):
    preprocessed=[]
    kernel = np.ones((3,3),np.uint8)
    for i in range(np.shape(array)[0]):
        image=cv2.cvtColor(array[i], cv2.COLOR_BGR2GRAY)
        img = image.copy()
        thresh_val, img_th = cv2.threshold(img, 240, 255, cv2.THRESH_BINARY_INV)
        img_th = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)
        img_th = cv2.erode(img_th,kernel,iterations = 2)
        img_th2 = img_th
        valid_range = np.logical_and(img_th,img_th2)
        img[np.logical_not(valid_range)] =0
        preprocessed.append(img)
    return preprocessed

In [4]:
def preprocessing_shape(array):
    preprocessed=[]
    for i in range(np.shape(array)[0]):
        img=cv2.cvtColor(array[i], cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(img, 240, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        preprocessed.append(thresh)
    return preprocessed

In [5]:
def preprocessing_color(array):
    preprocessed=[]
    kernel = np.ones((3,3),np.uint8)
    
    for i in range(np.shape(array)[0]):
        img = array[i].copy()
        threshold = [250,250,255]

        red = (threshold[2] > img[:,:,2])
        green = (threshold[1] > img[:,:,1])
        blue = (threshold[0] > img[:,:,0])

        valid_range = np.logical_and(red,green,blue)
        img[np.logical_not(valid_range)] = 0
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        img = cv2.erode(img,kernel,iterations = 3)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        preprocessed.append(img)
    return preprocessed

<h1>4. Deklarasi Fungsi Ekstraksi Fitur</h1>

In [6]:
def extract_texture(img):
    radius = 1
    n_points = 8 * radius
    
    lbp = local_binary_pattern(img, n_points, radius, method='uniform')
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins))
    hist = hist.astype("float")                         
    hist /= (hist.sum() + (1e-7))
    return hist

In [7]:
def extract_shape(img):
    moments = cv2.moments(img)
    huMoment = cv2.HuMoments(moments)
    for i in range(0,7):
        huMoment[i] = -1* np.copysign(1.0, huMoment[i]) * np.log10(abs(huMoment[i]))
    return huMoment.flatten()

In [8]:
def extract_color(img):
    features = []

    hist = cv2.calcHist([img], [0], None, [24], [0, 180])
                        
    return np.array(hist).flatten()

In [9]:
def featureextraction_texture(array):
    arr_feature = []
    for i in range(np.shape(array)[0]):
        arr_feature.append(extract_texture(array[i]))
    return arr_feature

In [10]:
def featureextraction_shape(array):
    arr_feature = []
    for i in range(np.shape(array)[0]):
        arr_feature.append(extract_shape(array[i]))
    return arr_feature

In [11]:
def featureextraction_color(array):
    arr_feature = []
    for i in range(np.shape(array)[0]):
        arr_feature.append(extract_color(array[i]))
    return arr_feature

In [55]:
X_train, y_train = load_dataset("Dataset2/Train/")

In [56]:
X_train_texture = preprocessing_texture(X_train)

In [57]:
X_train_shape = preprocessing_shape(X_train)

In [58]:
X_train_color = preprocessing_color(X_train)

In [59]:
X_train_texture_feature = featureextraction_texture(X_train_texture)

In [60]:
X_train_shape_feature = featureextraction_shape(X_train_shape)

In [94]:
X_train_color_feature = featureextraction_color(X_train_color)

In [62]:
clf1 = RandomForestClassifier(n_estimators=100, random_state=0)
clf1.fit(X_train_texture_feature, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [63]:
clf2 = RandomForestClassifier(n_estimators=100, random_state=0)
clf2.fit(X_train_shape_feature, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [95]:
clf3 = RandomForestClassifier(n_estimators=100, random_state=0)
clf3.fit(X_train_color_feature, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [65]:
X_train_texture_shape = np.concatenate((X_train_texture_feature,X_train_shape_feature),axis=1)

In [66]:
clf4 = RandomForestClassifier(n_estimators=100, random_state=0)
clf4.fit(X_train_texture_shape, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [96]:
X_train_texture_color = np.concatenate((X_train_texture_feature,X_train_color_feature),axis=1)

In [97]:
clf5 = RandomForestClassifier(n_estimators=100, random_state=0)
clf5.fit(X_train_texture_color, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [98]:
X_train_shape_color = np.concatenate((X_train_shape_feature,X_train_color_feature),axis=1)

In [99]:
clf6 = RandomForestClassifier(n_estimators=100, random_state=0)
clf6.fit(X_train_shape_color, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [100]:
combined_train_feature = np.concatenate((X_train_texture_feature,X_train_shape_feature,X_train_color_feature),axis=1)

In [101]:
clf7 = RandomForestClassifier(n_estimators=100, random_state=0)
clf7.fit(combined_train_feature, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [73]:
end1 = time.time()
print("Total running time of training scheme : ",end1-start1," second")

Total running time of training scheme :  3052.5215470790863  second


<h1> TEST SCHEME </h1>

In [74]:
start2 = time.time()

In [75]:
X_test, y_test = load_dataset("Dataset2/Test/")

In [76]:
X_test_texture = preprocessing_texture(X_test)

In [77]:
X_test_shape = preprocessing_shape(X_test)

In [78]:
X_test_color = preprocessing_color(X_test)

In [79]:
X_test_texture_feature = featureextraction_texture(X_test_texture)

In [80]:
X_test_shape_feature = featureextraction_shape(X_test_shape)

In [102]:
X_test_color_feature = featureextraction_color(X_test_color)

In [82]:
result1 = clf1.predict(X_test_texture_feature) 
print("Akurasi TEKSTUR : ", accuracy_score(y_test, result1) * 100)

Akurasi TEKSTUR :  61.35330170027679


In [83]:
result2 = clf2.predict(X_test_shape_feature) 
print("Akurasi BENTUK : ", accuracy_score(y_test, result2)*100)

Akurasi BENTUK :  50.06425464610518


In [104]:
result3 = clf3.predict(X_test_color_feature) 
print("Akurasi WARNA : ", accuracy_score(y_test, result3) * 100)

Akurasi WARNA :  70.1067615658363


In [85]:
X_test_texture_shape = np.concatenate((X_test_texture_feature,X_test_shape_feature),axis=1)

In [86]:
result4 = clf4.predict(X_test_texture_shape) 
print("Akurasi TEKSTUR + BENTUK : ", accuracy_score(y_test, result4) * 100)

Akurasi TEKSTUR + BENTUK :  78.98873072360617


In [105]:
X_test_texture_color = np.concatenate((X_test_texture_feature,X_test_color_feature),axis=1)

In [106]:
result5 = clf5.predict(X_test_texture_color) 
print("Akurasi TEKSTUR + WARNA : ", accuracy_score(y_test, result5) * 100)

Akurasi TEKSTUR + WARNA :  93.1395808620008


<h4>8.2.3 Klasifikasi Bentuk + Warna</h4>

In [107]:
X_test_shape_color = np.concatenate((X_test_shape_feature,X_test_color_feature),axis=1)

In [108]:
result6 = clf6.predict(X_test_shape_color) 
print("Akurasi BENTUK + WARNA : ", accuracy_score(y_test, result6) * 100)

Akurasi BENTUK + WARNA :  88.70601028074337


In [109]:
combined_test_feature = np.concatenate((X_test_texture_feature,X_test_shape_feature,X_test_color_feature),axis=1)

In [112]:
result7 = clf7.predict(combined_test_feature) 
print("Akurasi GABUNGAN : ", accuracy_score(y_test, result7) * 100)

Akurasi GABUNGAN :  94.34064847765914


In [51]:
end2 = time.time()
print("Total running time of test scheme : ",end2-start2," second")

Total running time of test scheme :  307.8309528827667  second


<h1> OVERALL RUNNING TIME </h1>

In [52]:
print("Total running time of all scheme : ",end2-start1," second")

Total running time of all scheme :  650.378308057785  second


In [53]:
start3 = time.time()

<h1> Validation Scheme </h1>

In [54]:
def randomize(arrayA,arrayB):
    randPerm = np.random.permutation(np.shape(arrayA)[0])
    newArrayA = []
    newArrayB = []
    for i in range(0,len(randPerm)):
        newArrayA.append(arrayA[randPerm[i]])
        newArrayB.append(arrayB[randPerm[i]])
    return np.array(newArrayA),np.array(newArrayB)

In [55]:
def KFoldCrossValidation(K,X,y):
    scores = []
    X,y = randomize(X,y)
    clf = RandomForestClassifier(n_estimators=100,random_state=0)
    cv = KFold(n_splits=K)
    for train_index, test_index in cv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf.fit(X_train, y_train)
        result = clf.predict(X_test)
        scores.append(accuracy_score(y_test, result))
        
    return np.mean(scores)

In [56]:
validasi1 = KFoldCrossValidation(3,combined_train_feature,y_train)
print("Akurasi VALIDASI GABUNGAN: ",validasi1*100," %")

Akurasi VALIDASI GABUNGAN:  99.8048399687744  %


In [57]:
validasi2 = KFoldCrossValidation(3,X_train_texture_feature,y_train)
print("Akurasi VALIDASI TEKSTUR: ",validasi2*100," %")

Akurasi VALIDASI TEKSTUR:  81.10200364298726  %


In [58]:
validasi3 = KFoldCrossValidation(3,X_train_shape_feature,y_train)
print("Akurasi VALIDASI SHAPE: ",validasi3*100," %")

Akurasi VALIDASI SHAPE:  83.50084569346863  %


In [59]:
validasi4 = KFoldCrossValidation(3,X_train_color_feature,y_train)
print("Akurasi VALIDASI WARNA: ",validasi4*100," %")

Akurasi VALIDASI WARNA:  90.36397345823576  %


In [60]:
validasi5 = KFoldCrossValidation(3,X_train_texture_shape,y_train)
print("Akurasi VALIDASI TEKSTUR+BENTUK: ",validasi5*100," %")

Akurasi VALIDASI TEKSTUR+BENTUK:  97.23360655737706  %


In [61]:
validasi6 = KFoldCrossValidation(3,X_train_texture_color,y_train)
print("Akurasi VALIDASI TEKSTUR+WARNA: ",validasi6*100," %")

Akurasi VALIDASI TEKSTUR+WARNA:  99.37386156648452  %


In [62]:
validasi7 = KFoldCrossValidation(3,X_train_shape_color,y_train)
print("Akurasi VALIDASI BENTUK+WARNA: ",validasi7*100," %")

Akurasi VALIDASI BENTUK+WARNA:  99.14454852979443  %


In [63]:
end3 = time.time()

In [64]:
print("Total running time of validation scheme : ",end3-start3," second")

Total running time of validation scheme :  494.83908009529114  second
